In [2]:
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

c:\asm\continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
mnist =  input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

In [5]:
num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

X = tf.placeholder(tf.float32, [None, timesteps, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [6]:
weights ={
    'out':tf.Variable(tf.random_normal([num_hidden, num_classes]))
}

biases = {
    'out':tf.Variable(tf.random_normal([num_classes]))
}

In [7]:
def RNN(x, weights, biases):
    x = tf.unstack(x, axis=1)
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    return tf.matmul(outputs[-1], weights['out']) +biases['out']

In [8]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(training_steps):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
    if step%display_step ==0:
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x, Y:batch_y})
        print("Step, Minibatch loss, accuracy: {}, {}, {}".format(step, loss, acc))
        
print("Optimization finished")
test_len = 1024
test_x = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
test_y = mnist.test.labels[:test_len]
loss, acc = sess.run([loss_op, accuracy], feed_dict={X:test_x, Y:test_y})
print("Test loss, accuracy: {}, {}, {}".format(step, loss, acc))
sess.close()

Step, Minibatch loss, accuracy: 0, 2.416642665863037, 0.09375
Step, Minibatch loss, accuracy: 200, 0.26644864678382874, 0.9296875
Step, Minibatch loss, accuracy: 400, 0.19935303926467896, 0.953125
Step, Minibatch loss, accuracy: 600, 0.1240614652633667, 0.953125
Step, Minibatch loss, accuracy: 800, 0.05916508287191391, 0.9765625
Step, Minibatch loss, accuracy: 1000, 0.13432404398918152, 0.953125
Step, Minibatch loss, accuracy: 1200, 0.09173615276813507, 0.9609375
Step, Minibatch loss, accuracy: 1400, 0.045294247567653656, 0.984375
Step, Minibatch loss, accuracy: 1600, 0.025582415983080864, 0.9921875
Step, Minibatch loss, accuracy: 1800, 0.050292838364839554, 0.9765625
Step, Minibatch loss, accuracy: 2000, 0.052292101085186005, 0.9765625
Step, Minibatch loss, accuracy: 2200, 0.07776615768671036, 0.96875
Step, Minibatch loss, accuracy: 2400, 0.03767552599310875, 0.984375
Step, Minibatch loss, accuracy: 2600, 0.00667452160269022, 1.0
Step, Minibatch loss, accuracy: 2800, 0.019533699378371

<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x000001C199272D68>>

In [10]:
sess.close()